In [1]:
# import some libs
import pandas as pd

In [245]:
# import main code lookup
mainsas = pd.read_excel('7001479 Japan Local Protocol covariates code list formatted for SAS v1.0.xlsx', index_col=None, sheet_name='Covariates for PS-match')

In [247]:
# data cleaning
mainsas['cov'] = mainsas['Covariate'].str.rstrip()

In [248]:
# import code requirement
goalsas = pd.read_excel('NBI covariates name .xlsx', index_col=None, sheet_name='Diabetes complications')

In [249]:
# data cleaning
goalsas['cov'] = goalsas['Covariates in table shells '].str.rstrip()

In [250]:
# merge to get lookup info
mergesas = mainsas.merge(goalsas, how='inner', left_on='cov', right_on='cov')

In [251]:
# generate sas diseasecode
dc = mergesas[mergesas['Variable'].str.contains("Diseasecode")]
for i in dc.itertuples():
    print('***', i[7],';')
    print('if', i[3], 'in', '(', i[4], ')', 'then', i[9]+'_dx_flag', '= 1;')
    print('else', i[9]+'_dx_flag', '= 0;')
    print()

*** Diabetic retinopathy ;
if Diseasecode in ( 8844537, 8830045, 8844536, 8830033, 2504010, 2504005, 2504013, 8843982, 8843990, 8844089
 ) then Diabetic_retinopathy_dx_flag = 1;
else Diabetic_retinopathy_dx_flag = 0;

*** Diabetes with other ophthalmic manifestations ;
if Diseasecode in ( 8845049, 8838065, 8845078, 8841692, 8841682, 
2504006 ) then Diabetes_WOOM_dx_flag = 1;
else Diabetes_WOOM_dx_flag = 0;

*** Diabetic foot ;
if Diseasecode in ( 8845068, 8845097, 8848634,8848633 ) then Diabetic_foot_dx_flag = 1;
else Diabetic_foot_dx_flag = 0;

*** Gangrene ;
if Diseasecode in ( 8843105, 8843106, 8848632,  2506006 ) then Gangrene_dx_flag = 1;
else Gangrene_dx_flag = 0;

*** Diabetes with unspecified complication ;
if Diseasecode in ( 2507028 ) then Diabetes_WUC_dx_flag = 1;
else Diabetes_WUC_dx_flag = 0;



In [252]:
# generate sas receiptcode
rc = mergesas[mergesas['Variable'].str.contains("Receiptcode")]
for i in rc.itertuples():
    print('***', i[7],';')
    print('if', i[3], 'in', '(', i[4], ')', 'then', i[9]+'_rc_flag', '= 1;')
    print('else', i[9]+'_rc_flag', '= 0;')
    print()

*** Retinal laser coagulation therapy ;
if Receiptcode in ( 150244110, 150244210 ) then Retinal_LT_therapy_rc_flag = 1;
else Retinal_LT_therapy_rc_flag = 0;

*** Lower extremity amputation ;
if Receiptcode in ( 150051610, 150051710, 150051810 ) then Lower_extremity_amputation_rc_flag = 1;
else Lower_extremity_amputation_rc_flag = 0;



In [253]:
# generate sas icd10 code
icd = mergesas[mergesas['Variable'].str.contains('ICD10code')]

def split(x, length):    
    splitted_string = x.split(',')
    result = [substr.strip() for substr in splitted_string if len(substr.strip())==length]
    result = ', '.join(result)
    return result

for length in [5, 6, 7]:
    icd[length] = icd['Code containing'].apply(lambda x: split(x, length))

C:\Users\guzhao\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [254]:
for i in icd.itertuples():
    print('***', i[7],';')
    if i[10] != '' and i[11] != '':
        print('if icd3 in', '(', i[10], ')', 'or', 'icd4 in', '(', i[11], ')', 'then', i[9]+'_flag', '= 1;')
        print('else', i[9]+'_rc_flag', '= 0;')
    elif i[10] == '' and i[11] != '' and i[12] =='':
        print('if icd4 in', '(', i[11], ')', 'then', i[9]+'_flag', '= 1;')
        print('else', i[9]+'_rc_flag', '= 0;')
    elif i[10] != '' and i[11] == '' and i[12] =='':
        print('if icd3 in', '(', i[10], ')', 'then', i[9]+'_flag', '= 1;')
        print('else', i[9]+'_rc_flag', '= 0;')
    elif i[10] == '' and i[11] == '' and i[12] !='':
        print('if icd5 in', '(', i[12], ')', 'then', i[9]+'_flag', '= 1;')
        print('else', i[9]+'_rc_flag', '= 0;')
    print()

*** Retinal detachment, vitreous hemorrhage, vitrectomy ;
if icd3 in ( "H33" ) or icd4 in ( "H431", "H450" ) then RD_VH_vitrectomy_flag = 1;
else RD_VH_vitrectomy_rc_flag = 0;

*** Diabetic neuropathy ;
if icd4 in ( "E104", "E114", "E134" ) then Diabetic_neuropathy_flag = 1;
else Diabetic_neuropathy_rc_flag = 0;

*** Diabetic nephropathy ;
if icd4 in ( "E102", "E112", "E132" ) then Diabetic_nephropathy_flag = 1;
else Diabetic_nephropathy_rc_flag = 0;

*** Hypoglycemia ;
if icd4 in ( "E160", "E161", "E162", "T383" ) then Hypoglycemia_flag = 1;
else Hypoglycemia_rc_flag = 0;

*** Hypoglycemia ;
if icd4 in ( "E106", "E116" ) then Hypoglycemia_flag = 1;
else Hypoglycemia_rc_flag = 0;

*** Hyperglycemia ;
if icd4 in ( "R739" ) then Hyperglycemia_flag = 1;
else Hyperglycemia_rc_flag = 0;

*** Disorders of fluid electrolyte and acid-base balance ;
if icd3 in ( "E86", "E87" ) then Disorders_FE_ADD_flag = 1;
else Disorders_FE_ADD_rc_flag = 0;

*** Diabetic ketoacidosis ;
if icd4 in ( "E101", "E